In this notebook, we are going to model a transformer(Time Series Transformer). Unlike the LSTMs, they don't understand "Order" naturally and we must inject positional encodings so the model knows the sequence order.